<a href="https://colab.research.google.com/github/irisvanwalraven/odcm/blob/main/KAYAK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <span style="color:orange">Scraping data from KAYAK</span>

For the course Online Data Collection & Management at Tilburg University, our team aims to scrape data from the website kayak.com. KAYAK provides a service that compares airline tickets with one another. When on the explore page, KAYAK provides a set of destinations based on the airport of departure. The purpose of this scraper is to create a dataset in which the suggested destinations based on the airport of departure (the top 10 biggest airports in Europe) become visible in April for a trip duration of five days.

## Top 10: Biggest airports in Europe
The top 10 of the biggest airports in Europe is as follows:
1. London Heathrow Airport, United Kingdom (LHA)
2. Aéroport de Paris-Charles de Gaulle, France (CDG)
3. Amsterdam Airport Schiphol, the Netherlands (AMS)
4. Flughafen Frankfurt am Main, Germany (FRA)
5. Aeropuerto Adolfo Suárez Madrid-Barajas, Spain (MAD)
6. Aeroport de Barcelona-el Prat, Spain (BCN)
7. Istanbul Airport, Turkey (IST)
8. Sheremetyevo International Airport, Russia (SVO)
9. Flughafen München Franz Josef Strauß, Germany (MUC)
10. London Gatwick Airport, United Kingdom (LGW)

In this notebook we will discuss the scraping in three chapters:
1. The preparation before scraping
2. The kayak.com/explore scraper
3. Saving the scraped data in a csv file

# <span style="color:orange">1. The preparation before scraping</span>
Before the explore page of kayak can be scraped a few things need to be done. The libraries need to be imported for the code to run. 

## 1.1 Importing libraries
In the first cell the necessary libraries for running our code are imported. This are the libraries we need:
* The requests library lets us make HTML request to Kayak's website server for retrieving the data from their page.
* The BeautifulSoup library is easy to use for beginners and will allow us to extract data from HTML files.
* The time library will be used to pause the execution of the commands. We use this because of the amount of data we gather. 
* The csv library will be used to store the scraped data in a csv file in the end.
* The datetime library is necessary to add the current day to our csv file.


In [ ]:
import requests
from bs4 import BeautifulSoup
import time
from time import sleep 
import csv
from datetime import datetime


In [ ]:
# To use Chromedriver on Google Colab, run this code
!pip install selenium
!apt-get update 
!apt install chromium-chromedriver

from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

     |████████████████████████████████| 968 kB 12.7 MB/s 
     |████████████████████████████████| 138 kB 67.3 MB/s 
     |████████████████████████████████| 359 kB 68.0 MB/s 
     |████████████████████████████████| 3.6 MB 64.4 MB/s 
     |████████████████████████████████| 55 kB 3.0 MB/s 
     |████████████████████████████████| 58 kB 5.1 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.9 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:4 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:6 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:7 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Get:8 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:9 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:13 https://

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: use options instead of chrome_options
  # This is added back by InteractiveShellApp.init_path()


In [ ]:
# To use Chromedriver on your computer, run this code
import selenium.webdriver

driver = selenium.webdriver.Chrome()

WebDriverException: ignored

# <span style="color:orange">2. The kayak.com/explore scraper</span>


### 2.1 Define airports and extract airport urls

In [ ]:
# Defining base url
base_url = 'https://www.kayak.com/explore'

# Create a list of all the airports we want to extract data from
airports = ["LHR", "CDG", "AMS", "FRA", "MAD", "BCN", "IST", "SVO", "MUC", "LGW"]

# Loop for airport urls
airport_urls = [] # Make an empty list to save the airport urls

for airport in airports:
    airport_url = 'https://www.kayak.com/explore/' + airport + '-anywhere/20220401,20220430' + '?tripdurationrange=5,5'
    airport_urls.append(airport_url) 
airport_urls # List of urls where we want to extract data from

['https://www.kayak.com/explore/LHR-anywhere/20220401,20220430?tripdurationrange=5,5',
 'https://www.kayak.com/explore/CDG-anywhere/20220401,20220430?tripdurationrange=5,5',
 'https://www.kayak.com/explore/AMS-anywhere/20220401,20220430?tripdurationrange=5,5',
 'https://www.kayak.com/explore/FRA-anywhere/20220401,20220430?tripdurationrange=5,5',
 'https://www.kayak.com/explore/MAD-anywhere/20220401,20220430?tripdurationrange=5,5',
 'https://www.kayak.com/explore/BCN-anywhere/20220401,20220430?tripdurationrange=5,5',
 'https://www.kayak.com/explore/IST-anywhere/20220401,20220430?tripdurationrange=5,5',
 'https://www.kayak.com/explore/SVO-anywhere/20220401,20220430?tripdurationrange=5,5',
 'https://www.kayak.com/explore/MUC-anywhere/20220401,20220430?tripdurationrange=5,5',
 'https://www.kayak.com/explore/LGW-anywhere/20220401,20220430?tripdurationrange=5,5']

### 2.2 HTLM code of the city names, country names and prices

Code of the header where to get the information from: id = "JoPq-destinations" and class = "_ihz _ilc _iai">
# <div id="JoPq-destinations" class="_ihz _ilc _iai">

City name code: "_ib0 _igh _ial _1O _iaj City__Name"
For example: <div class="_ib0 _igh _ial _1O _iaj City__Name">Bucharest</div>

Country name code class = "_iC8 _1W _ib0 _iYh _igh Country__Name"
For example: <div class="_iC8 _1W _ib0 _iYh _igh Country__Name">Romania</div>

Prices code: "_ib0 _18 _igh _ial _iaj"
For example: <div class="_ib0 _18 _igh _ial _iaj">from $52</div>

These HTML codes will be used in the next step

### 2.3 Extract data from urls

In [ ]:
# From airport_urls we want to scrape the 16 destinations and prices per url
destination_city=[]
destination_country=[]
destination_price=[]2
driver.get('https://www.kayak.com/explore/' + airport + '-anywhere/20220401,20220430' + '?tripdurationrange=5,5')

In [ ]:
id = driver.find_elements_by_class_name('_ihz _ilc _iai')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: find_elements_by_class_name is deprecated. Please use find_elements(by=By.CLASS_NAME, value=name) instead
  """Entry point for launching an IPython kernel.


In [ ]:
# Code to sum up all the city names, country names and prices per airport_url
content = driver.page_source
soup = BeautifulSoup(content)
for a in soup.findAll('id',href=True, attrs={'class':'_ihz _ilc _iai'}):
city=id.find('div', attrs={'class':'_ib0 _igh _ial _1O _iaj City__Name'})
country=id.find('div', attrs={'class':'_iC8 _1W _ib0 _iYh _igh Country__Name'})
price=id.find('div', attrs={'class':'_ib0 _18 _igh _ial _iaj'})

city.append(city.text) # To get the city names
country.append(country.text) # To get the country names
price.append(prices.text) # To get the prices

# Create a dataframe with all the information scraped from the urls
df = pd.DataFrame({'City':cities,'countries':countries,'Price':prices})

# <span style="color:orange">3. Saving the scraped data in a csv file</span>
